<a href="https://colab.research.google.com/github/poornima15-10/poornima13/blob/main/image_captioning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -uq "/content/drive/My Drive/Flickr8k_text.zip" -d "/content/drive/My Drive/Flickr8k_text"

In [ ]:
file_data = open("/content/drive/My Drive/Flickr8k_text/Flickr8k.token.txt",'r')
image_files = []
captions = []
for line in file_data.readlines():
    spl = line.split("#")
    image_files.append(spl[0])
    captions.append(str(spl[1]))
del file_data

In [ ]:
img_cap_dict = {"images":image_files,"captions":captions}


In [ ]:
import pandas as pd
img_cap_df = pd.DataFrame(img_cap_dict,columns=img_cap_dict.keys())

In [ ]:
img_cap_df.head()

In [ ]:
# prepare translation table for removing punctuation
import string
table = str.maketrans('', '', string.punctuation)
def clean_captions(caption):
    # tokenize
    caption = caption.split()
    # convert to lower case
    caption = [word.lower() for word in caption]
    # remove punctuation from each token
    caption = [w.translate(table) for w in caption]
    # remove hanging 's' and 'a'
    caption = [word for word in caption if len(word)>1]
    # remove tokens with numbers in them
    caption = [word for word in caption if word.isalpha()]
    # store as string
    return ' '.join(caption)

In [ ]:
img_cap_df["captions"] = img_cap_df["captions"].apply(lambda x:clean_captions(x))

In [ ]:
img_cap_df["captions"].values[:10]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!unzip -uq "/content/drive/My Drive/Flickr8k_Dataset.zip" -d "/content/drive/My Drive/Flickr8k_Dataset"

In [ ]:
import numpy as np
from numpy import array

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

import string
import os
from PIL import Image
import glob
from pickle import dump, load
from nltk.translate.bleu_score import corpus_bleu

from tensorflow.keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Embedding, Bidirectional, TimeDistributed, Dense, RepeatVector, Activation, Flatten, Reshape, concatenate, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras import Input, layers
from tensorflow.keras import optimizers
from tensorflow.keras.layers import add
from tensorflow.keras.utils import plot_model
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping,ReduceLROnPlateau

from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [ ]:
print(img_cap_df["images"].values[np.random.randint(img_cap_df.shape[0])])

In [ ]:
import numpy as np
img_file = img_cap_df["images"].values[np.random.randint(img_cap_df.shape[0])]
img_caption = img_cap_df["captions"][img_cap_df["images"] == img_file]

x=plt.imread("/content/drive/My Drive/Flickr8k_Dataset/Flicker8k_Dataset/"+img_file)
plt.imshow(x)
plt.show()
print("############## Image Description ###############\n")
print(img_caption)
